In [1]:
from chebgreen import ChebGreen
import numpy as np
import matplotlib.pyplot as plt

from chebgreen.chebpy2 import Quasimatrix
from chebgreen.chebpy2.chebpy import chebfun
from chebgreen.chebpy2 import Chebfun2

Loading settings from /home/admin-quad/harsh/chebgreen/examples/settings.ini.

ChebGreen settings:
[MATLAB]
nsample = 100
lambda = 0.01
nf = 500
nu = 500
noise = 0

[GREENLEARNING]
trainratio = 0.95
epochs_adam = 5000
epochs_lbfgs = 0
initlearningrate = 0.01
finallearningrate = 0.001
layerconfig = [50,50,50,50]
activation = rational

[GENERAL]
precision = 64
device = cpu

MATLAB Path = /usr/local/MATLAB/R2022b/bin/matlab
PyTorch device = cpu


In [2]:
%%time
Theta = [0.8,0.9,0.95]
theta_ = 0.85
domain = [-np.pi/2,np.pi/2,-np.pi/2,np.pi/2]
generateData = True
script = "generate_fractional"
example = "fractional_laplacian"
dirichletBC = False

CPU times: user 9 μs, sys: 0 ns, total: 9 μs
Wall time: 14.8 μs


In [3]:
%%time
model = ChebGreen(Theta, domain, generateData, script, example, dirichletBC)

Generating dataset for example 'fractional_laplacian'
Dataset found for Theta = 0.80. Skipping dataset generation.
Dataset found for Theta = 0.90. Skipping dataset generation.
Dataset found for Theta = 0.95. Skipping dataset generation.
-------------------------------------------------------------------------------

Generating chebfun2 models:
Found saved model, Loading model for example 'fractional_laplacian' at Theta = 0.80
Learning a chebfun model for example 'fractional_laplacian' at Theta = 0.80


RuntimeWarning: CHEBFUN:CHEBFUN2:constructor:rank, Not a low-rank function.

In [4]:
model.G

NameError: name 'model' is not defined

In [ ]:
model.N

In [ ]:
model.G[0.8]

In [ ]:
%%time
for t in Theta:
    fig = plt.figure()
    model.G[t].plot(fig = fig)
    error = model.computeEmpiricalError(t)
    plt.title(f"$\\theta = {{{t:4.2f}}}$,$\\epsilon_{{test}} = {{{error*100:4.2f}}}\\%$")

In [ ]:
# vmin, vmax = -0.44,2.8
# Gplot = model.G[0.95]

# xx = np.linspace(domain[0],domain[1],2000)
# yy = np.linspace(domain[2],domain[3],2000)
# x, y = np.meshgrid(xx,yy)
# Gplot = Gplot[x,y]

# fig = plt.figure(figsize = (13,10), frameon=False)
# plt.axis('off')
# plt.gca().set_aspect('equal', adjustable='box')
# levels = np.linspace(vmin, vmax, 50, endpoint = True)
# plt.contourf(x,y,Gplot, levels = levels, cmap = 'jet', vmin = vmin, vmax = vmax)
# # ticks = np.linspace(vmin, vmax, 10, endpoint=True)
# # cbar = plt.colorbar(ticks = ticks, modeltion = 0.046, pad = 0.04)

# fig.savefig('model-C.png', dpi = fig.dpi, bbox_inches='tight', pad_inches=0)

In [ ]:
%%time
u,s,v = model.G[Theta[0]].svd()

In [ ]:
fig = plt.figure(figsize = (8,6))
plt.plot(s)
plt.yscale('log')
plt.grid(True)

In [ ]:
# i = slice(10,20)
i = slice(0,5)
# i = 16
fig = plt.figure(figsize = (8,6))
u[:,i].plot(fig = fig)
# print(np.max(u[:,i].vscale))
# plt.title(f'First {i+1} left singular functions for a Laplacian')
plt.xlim([0,1])
# fig.savefig(f'Laplace-first.png', dpi = fig.dpi)

In [ ]:
%%time
Ginterp, Ninterp = model.generateNewModel(theta_)

In [ ]:
validation = ChebGreen([theta_], domain, generateData, script, example, dirichletBC)

In [ ]:
fig = plt.figure()
Ginterp.plot(fig = fig)
error = model.computeEmpiricalError(theta_)
plt.title(f"Interpolated Model $\\theta = {{{theta_:4.2f}}}$,$\\epsilon_{{test}} = {{{error*100:4.2f}}}\\%$")

In [ ]:
fig = plt.figure()
validation.G[theta_].plot(fig = fig)
error = validation.computeEmpiricalError(theta_)
plt.title(f"Validation Model $\\theta = {{{theta_:4.2f}}}$,$\\epsilon_{{test}} = {{{error*100:4.2f}}}\\%$")

In [ ]:
def compareGreen1D(interpModel, exactModel, vmin = None, vmax = None):
    domain = interpModel.domain
    xx = np.linspace(domain[2],domain[3],2000)
    yy = np.linspace(domain[0],domain[1],2000)
    x, s = np.meshgrid(xx,yy)
    
    if vmin is None:
        vmin = min(interpModel.min,exactModel.min)
    if vmax is None:
        vmax = max(interpModel.max,exactModel.max)
        
    plt.figure(figsize = (16,6))
    plt.subplot(121)
    
    GInterp = interpModel[x,s]
    
    levels = np.linspace(vmin, vmax, 50, endpoint = True)
    ticks = np.linspace(vmin, vmax, 10, endpoint=True)
    
    surf = plt.contourf(x, s, GInterp, levels = levels, cmap = 'turbo', vmin = vmin, vmax = vmax)
    plt.gca().set_aspect('equal', adjustable='box')
    cbar = plt.colorbar(ticks = ticks, fraction = 0.046, pad = 0.04)
    cbar.ax.set_yticklabels(["{:4.2f}".format(i) for i in ticks])
    plt.xlabel('$x$', fontsize = 11)
    plt.ylabel('$s$', rotation='horizontal', labelpad=10, fontsize = 11)
    plt.title("Interpolated", fontsize = 14)
    
    GExact = exactModel[x,s]
    
    plt.subplot(122)
    plt.gca().set_aspect('equal', adjustable='box')
    surf = plt.contourf(x, s, GExact, levels = levels, cmap = 'turbo', vmin = vmin, vmax = vmax)
    cbar = plt.colorbar(ticks = ticks, fraction = 0.046, pad = 0.04)
    cbar.ax.set_yticklabels(["{:4.2f}".format(i) for i in ticks])
    plt.xlabel('x', fontsize = 11)
    plt.ylabel('s', rotation='horizontal', labelpad=10, fontsize = 11)
    plt.title("Data", fontsize = 14)

In [ ]:
compareGreen1D(Ginterp,validation.G[theta_])

In [ ]:
from chebgreen.backend import config
from chebgreen.greenlearning.quadrature_weights import get_weights
from chebgreen.greenlearning.utils import DataProcessor
from copy import deepcopy

theta = Theta[0]

data = DataProcessor(model.datapath + f"/{theta:.2f}/data.mat")
data.generateDataset(trainRatio = 0.95)

# Ef = data.valDataset[0]
# Eu = data.valDataset[1]
Ef = data.trainDataset[0].detach().cpu().numpy()
Eu = data.trainDataset[1].detach().cpu().numpy()

xF, xU = data.xF, data.xU
wF = get_weights('trapezoidal', xF).astype(dtype = config(np))
wU = get_weights('trapezoidal', xU).astype(dtype = config(np))

In [ ]:
plt.figure()
idx = 3
plt.plot(Ef[idx,:])